![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

In [1]:
import numpy as np
import pandas as pd

In [2]:
population = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_population.csv")
vegetal = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_vegetaux.csv")
animaux = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_animaux.csv")
cereals = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_céréales.csv")
under = pd.read_csv("/Users/katrinakonz/Desktop/OPENCLASSROOMS/P3/P3_Katrina_JUMADIAO/P3_01_qa/FAO_2013_fr/fr_sousalimentation.csv")

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Population

In [3]:
population.rename(columns= {"Code Domaine": "domain_code",
                            "Domaine": "domain",
                            "Code zone":"code_pays",
                            "Zone":"pays",
                            "Code Élément": "element_code",
                            "Élément": "element",
                            "Code Produit":"code_produit",
                            "Produit":"produit",
                            "Code année": "year_code",
                            "Année":"annee",
                            "Unité": "unite",
                            "Valeur": "population",
                            "Symbole": "symbol",
                            "Description du Symbole": "description"
                           }, inplace=True)

# Conversion des unités (1000 personnes)
population["population"] *= 1000

# Sans excès (Chine)
population.loc[population["pays"].str.contains("Chine", case=False)]
population.drop(index=33, inplace=True)

# Creation de la DB
population = population[["pays", "code_pays", "annee", "population"]].copy()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# dispo_alim et equilibre_prod

In [4]:
# Concatenation: animaux + vegetaux
animaux["origine"] = "animal"
vegetal["origine"] = "vegetal"
foods = pd.concat([animaux, vegetal])

# Pivot: animaux + vegetaux
foods = pd.pivot_table(foods, index=["Code zone","Zone","Code Produit","Produit","Année","origine"], values="Valeur", columns="Élément")
foods = foods.rename_axis(None, axis=1)
foods.reset_index(inplace=True)

# sans excès (Chine)
foods.drop(foods[foods["Zone"] == "Chine"].index, inplace=True)

# Remplacer les valeurs NaN par 0
foods.replace(np.nan, 0, inplace=True)

# Conversion des unités (milliers de tonnes)
tons_k = ["Aliments pour animaux","Autres utilisations (non alimentaire)","Disponibilité intérieure",
            "Exportations - Quantité","Importations - Quantité","Nourriture","Pertes","Production",
            "Semences","Traitement","Variation de stock"]

for list in tons_k :
    foods[list] *= 1000000
    
# Renommer les colonnes
foods.rename(columns= {"Code zone":"code_pays",
                       "Zone":"pays",
                       "Code Produit":"code_produit",
                       "Produit":"produit",
                       "Année":"annee",
                       "Aliments pour animaux":"animal_food",
                       "Disponibilité intérieure":"dispo_int",
                       "Disponibilité alimentaire en quantité (kg/personne/an)":"dispo_alim_tonnes",
                       "Disponibilité alimentaire (Kcal/personne/jour)":"dispo_alim_kcal_p_j",
                       "Disponibilité de protéines en quantité (g/personne/jour)":"dispo_prot",
                       "Disponibilité de matière grasse en quantité (g/personne/jour)":"dispo_mat_gr",
                       "Autres utilisations (non alimentaire)":"other_uses",
                       "Nourriture":"nourriture",
                       "Pertes":"pertes",
                       "Production":"production",
                       "Semences":"semences",
                       "Traitement":"transfo",
                       "Variation de stock":"stock_var"
                       }, inplace=True)

# Creation de la BD
dispo_alim = foods[["pays", "code_pays", "annee", "produit", "code_produit", "origine", "dispo_alim_tonnes", "dispo_alim_kcal_p_j", "dispo_prot", "dispo_mat_gr"]].copy()
equilibre_prod = foods[["pays", "code_pays", "annee", "produit", "code_produit", "dispo_int", "animal_food", "semences", "pertes", "transfo", "nourriture", "other_uses"]].copy()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# sous_nutrition

In [5]:
under.rename(columns= {"Code Domaine": "domain_code",
                       "Domaine": "domain",
                       "Code zone":"code_pays",
                       "Zone":"pays",
                       "Code Élément": "element_code",
                       "Élément": "element",
                       "Code Produit":"code_produit",
                       "Produit":"produit",
                       "Code année": "year_code",
                       "Année":"annee",
                       "Unité": "unit",
                       "Valeur": "nb_personnes",
                       "Symbole": "symbol",
                       "Description du Symbole": "description",
                       "Note": "note"
                       }, inplace=True)

# Trouver l'année 2013 et nettoyage des données
under["annee"] = pd.to_numeric(under["annee"].str[0:4])+1
under.replace('<0.1', 0.0, inplace=True)
under["nb_personnes"] = under["nb_personnes"].astype(float)

# Remplacer les valeurs NaN par 0
under.replace(np.nan, 0, inplace=True)

# Conversion des unités (millions)
under["nb_personnes"] *= 1000000

# Isolation de l'année 2013 sans excès (Chine)
under = under.loc[under["annee"] == 2013]
under.loc[under["pays"].str.contains("Chine", case=False)]
under.drop(index=185, inplace=True)

# Creation de la DB
sous_nutrition = under[["pays", "code_pays", "annee", "nb_personnes"]].copy()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
# Remplacer les lettres avec accents

In [6]:
population["pays"] = population["pays"].str.replace(u"ç", "c")
population["pays"] = population["pays"].str.replace(u"è", "e")
population["pays"] = population["pays"].str.replace(u"é", "e")
population["pays"] = population["pays"].str.replace(u"ê", "e")
population["pays"] = population["pays"].str.replace(u"ë", "e")
population["pays"] = population["pays"].str.replace(u"î", "i")
population["pays"] = population["pays"].str.replace(u"ï", "i")
population["pays"] = population["pays"].str.replace(u"ô", "o")
population["pays"] = population["pays"].str.replace(u"É", "E")
population["pays"] = population["pays"].str.replace(u"È", "E")
population["pays"] = population["pays"].str.replace(u"Ê", "E")
population["pays"] = population["pays"].str.replace(u"Ë", "E")
population["pays"] = population["pays"].str.replace(u"Ï", "I")
population["pays"] = population["pays"].str.replace(u"Î", "I")

dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"ç", "c")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"è", "e")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"é", "e")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"ê", "e")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"ë", "e")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"î", "i")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"ï", "i")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"ô", "o")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"É", "E")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"È", "E")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"Ê", "E")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"Ë", "E")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"Ï", "I")
dispo_alim["pays"] = dispo_alim["pays"].str.replace(u"Î", "I")

equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"ç", "c")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"è", "e")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"é", "e")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"ê", "e")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"ë", "e")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"î", "i")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"ï", "i")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"ô", "o")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"É", "E")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"È", "E")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"Ê", "E")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"Ë", "E")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"Ï", "I")
equilibre_prod["pays"] = equilibre_prod["pays"].str.replace(u"Î", "I")

sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"ç", "c")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"è", "e")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"é", "e")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"ê", "e")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"ë", "e")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"î", "i")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"ï", "i")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"ô", "o")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"É", "E")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"È", "E")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"Ê", "E")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"Ë", "E")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"Ï", "I")
sous_nutrition["pays"] = sous_nutrition["pays"].str.replace(u"Î", "I")

# --------------------------------------------------------------------
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"ç", "c")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"è", "e")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"é", "e")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"ê", "e")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"ë", "e")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"î", "i")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"ï", "i")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"ô", "o")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"É", "E")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"È", "E")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"Ê", "E")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"Ë", "E")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"Ï", "I")
dispo_alim["produit"] = dispo_alim["produit"].str.replace(u"Î", "I")

equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"ç", "c")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"è", "e")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"é", "e")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"ê", "e")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"ë", "e")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"î", "i")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"ï", "i")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"ô", "o")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"É", "E")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"È", "E")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"Ê", "E")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"Ë", "E")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"Ï", "I")
equilibre_prod["produit"] = equilibre_prod["produit"].str.replace(u"Î", "I")

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
# EXPORTING

In [7]:
population.to_csv("population.csv", index = False, encoding="latin1")
dispo_alim.to_csv("dispo_alim.csv", index = False, encoding="latin1")
equilibre_prod.to_csv("equilibre_prod.csv", index = False, encoding="latin1")
sous_nutrition.to_csv("sous_nutrition.csv", index = False, encoding="latin1")